In [153]:

import math
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils
import torch.utils.data
import torch.nn as nn
import torchsummary
import torch.nn.functional as F
import numpy as np
import random
import importlib
import gen_utils
importlib.reload(gen_utils)

<module 'gen_utils' from '/home/spometun/projects/solink/scripts/python/home_project/life/gen_utils.py'>

In [154]:
transform = transforms.Compose(
    [transforms.ToTensor()
        ,transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])
train=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=True,
                                  download=True, transform=transform)
test=torchvision.datasets.CIFAR10("/home/spometun/datasets/research", train=False,
                                  download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [158]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

set_seed(2)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
# torchsummary.summary(net, (3, 32, 32), device=torch.device("cpu"))

In [159]:
def get_whole_data(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset))
    data = next(iter(loader))
    return data

train_whole_data = get_whole_data(train)
test_whole_data = get_whole_data(test)

def calc_stats(network, data):
    with torch.no_grad():
        outputs = network(data[0])
    labels = data[1]

    total = len(labels)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    _loss = criterion(outputs, labels)
    print(f"Loss {_loss: .2f} Accuracy {100 * correct / total:.1f}%")



In [162]:
train_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True)

def criterion(input, target):
    return nn.functional.cross_entropy(input, target) / math.log(2)

optimizer = torch.optim.SGD(net.parameters(), lr=0.0002, momentum=0.9)
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 1):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i:5d}] running loss: {running_loss / 2000:.2f}')
            running_loss = 0.0
            calc_stats(net, train_whole_data)
            calc_stats(net, test_whole_data)

print('Finished Training')

[1,  2000] running loss: 1.30
Loss  1.24 Accuracy 69.4%
Loss  1.51 Accuracy 64.4%
[1,  4000] running loss: 1.24
Loss  1.21 Accuracy 70.3%
Loss  1.49 Accuracy 64.6%
[1,  6000] running loss: 1.27
Loss  1.22 Accuracy 70.1%
Loss  1.51 Accuracy 64.0%
[1,  8000] running loss: 1.26
Loss  1.17 Accuracy 71.4%
Loss  1.47 Accuracy 65.0%
[1, 10000] running loss: 1.25
Loss  1.17 Accuracy 71.3%
Loss  1.48 Accuracy 64.5%
[1, 12000] running loss: 1.24
Loss  1.14 Accuracy 72.2%
Loss  1.46 Accuracy 65.3%
[2,  2000] running loss: 1.16
Loss  1.14 Accuracy 72.1%
Loss  1.49 Accuracy 65.5%
[2,  4000] running loss: 1.19
Loss  1.15 Accuracy 71.9%
Loss  1.49 Accuracy 64.8%
[2,  6000] running loss: 1.18
Loss  1.14 Accuracy 72.0%
Loss  1.48 Accuracy 64.8%
[2,  8000] running loss: 1.15
Loss  1.11 Accuracy 72.8%
Loss  1.47 Accuracy 65.0%
[2, 10000] running loss: 1.16
Loss  1.11 Accuracy 72.4%
Loss  1.47 Accuracy 65.1%
[2, 12000] running loss: 1.18
Loss  1.10 Accuracy 73.0%
Loss  1.47 Accuracy 65.4%
Finished Trainin